In [80]:
import pandas as pd
import numpy as np

In [81]:
file_to_load = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_to_load)

In [82]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [83]:
# data cleaning 
purchase_data = purchase_data.dropna(how='any')
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [84]:
# players summary
player_summary = purchase_data.loc[:, ["Gender", "SN", "Age"]]
player_summary.head()


,Gender,SN,Age
0,Male,Lisim78,20
1,Male,Lisovynya38,40
2,Male,Ithergue48,24
3,Male,Chamassasya86,24
4,Male,Iskosia90,23


In [85]:
player_summary = player_summary.drop_duplicates()
num_players = player_summary.count()[0]
num_players

576

In [86]:
# unique items
num_unique_items = len(purchase_data["Item ID"].unique())
# average purchase price
ave_purchase_price = round(purchase_data["Price"].mean())
# total number purchase
total_num_purchases = purchase_data["Item Name"].count()
# total revenue
total_revenue = purchase_data["Price"].sum()

In [87]:
summary_df_player = pd.DataFrame({"Number of Unique Items":num_unique_items,
                           "Average Purchase Price":ave_purchase_price,
                           "Total Number of Purchases":total_num_purchases,
                           "Total Revenue":total_revenue},index = [0])

In [88]:
summary_df_player

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,3,780,2379.77


In [89]:
# gender demographics
gender_demographics_totals = player_summary["Gender"].value_counts()
gender_demographics_percents = gender_demographics_totals / num_players * 100
gender_demographics = pd.DataFrame({"Total Count": gender_demographics_totals,
                                    "Percentage of Players": gender_demographics_percents})

gender_demographics



,Total Count,Percentage of Players
Male,484,84.027778
Female,81,14.062500
Other / Non-Disclosed,11,1.909722


In [96]:
# Basic Calculations
gender_purchase_total = purchase_data.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_average = purchase_data.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Value")
gender_counts = purchase_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")


# Create a summary data frame to hold the results
ave_total_puchase_person = gender_purchase_total / gender_demographics["Total Count"]

#data cleaner formatting
gender_data = pd.DataFrame({"Purchase Count": gender_counts, 
                            "Average Purchase Value": gender_average, 
                           " avg. purchase total per person": ave_total_puchase_person})

# Display the summary data frame
gender_data

,Purchase Count,Average Purchase Value,avg. purchase total per person
Gender,,,
Female,113,3.203009,4.468395
Male,652,3.017853,4.065372
Other / Non-Disclosed,15,3.346000,4.562727


In [108]:
# Establish bins for ages
Ages_Bin = [0,10,15,20,25,30,35,40,100]
Ages_group = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
purchase_data['Age Ages_Bin'] = pd.cut(purchase_data['Age'],Ages_Bin,labels=Ages_group,right=False)

purchase_data_age_groupby = purchase_data.groupby(by='Age Ages_Bin')


#Use Cut to categorize players using the age bins. Hint: use pd.cut()
player_summary["Age Ranges"] = pd.cut(player_summary["Age"], Ages_Bin, labels=Ages_group)
player_summary.head()

,Gender,SN,Age,Age Ranges
0,Male,Lisim78,20,15-19
1,Male,Lisovynya38,40,35-39
2,Male,Ithergue48,24,20-24
3,Male,Chamassasya86,24,20-24
4,Male,Iskosia90,23,20-24


In [110]:
# Calculate the numbers and percentages by age group
age_demographics_totals = player_summary["Age Ranges"].value_counts()
age_demographics_percents = age_demographics_totals / num_players * 100

age_demographics = pd.DataFrame({"Total Count": age_demographics_totals, "Percent of Players": age_demographics_percents})
age_demographics = age_demographics.sort_index()

age_demographics

,Total Count,Percent of Players
<10,24,4.166667
10-14,41,7.118056
15-19,150,26.041667
20-24,232,40.277778
25-29,59,10.243056
30-34,37,6.423611
35-39,26,4.513889
40+,7,1.215278


In [111]:
# top spender
purchase_data_spender_groupby = purchase_data.groupby("SN")
total_purchase_value = purchase_data_spender_groupby["Price"].sum()
purchase_count = purchase_data_spender_groupby["Price"].count()
average_purchase_price = round(purchase_data_spender_groupby["Price"].mean(),2)

In [114]:
display_df_spender = pd.DataFrame({"Purchase Count": purchase_count,
                                   "Average Purchase Price":average_purchase_price,
                                   "Total Purchase Value":total_purchase_value})
                  
                  
display_df_spender.sort_values(by = 'Total Purchase Value', ascending=False).head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


In [125]:
# most popular 
purchase_data_popular_groupby = purchase_data.groupby(["Item ID", "Item Name"])
purchase_count = purchase_data_popular_groupby["Price"].count()
item_price = purchase_data_popular_groupby["Price"].mean()
total_purchase_price = purchase_data_popular_groupby["Price"].sum()

display_df_popular = pd.DataFrame({"Purchase Count":purchase_count,
                                    "Item Price": item_price,
                  
                                   "Total Purchase Price": total_purchase_price,
                  })
display_df_popular.sort_values(by="Purchase Count",ascending= False).head()

,,Purchase Count,Item Price,Total Purchase Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [128]:
# most profitable items
purchase_data_profitable_groupby = purchase_data.groupby(["Item ID", "Item Name"])
purchase_count = purchase_data_profitable_groupby["Price"].count()
item_price = purchase_data_profitable_groupby["Price"].mean()
total_purchase_price = purchase_data_profitable_groupby["Price"].sum()

display_df_profitable = pd.DataFrame({'Purchase Count':purchase_count,
                                      'Item Price': item_price,
                  
                                      'Total Purchase Price': total_purchase_price})
                  
display_df_profitable.sort_values(by="Total Purchase Price",ascending= False).head()

,,Purchase Count,Item Price,Total Purchase Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
